Strategy Optimization

In [1]:
import SMABacktester as SMA
import MeanRevBacktester as MeanRev
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("twenty_minutes.csv", parse_dates=["time"], index_col="time")
symbol = "EURUSD"
symbol

'EURUSD'

In [5]:
def optimal_strategy(parameters):
    start = "2019-01-01"
    end = "2020-08-30"
    symbol = "EURUSD"
    tc = 0.000059

    #SMA
    tester1 = SMA.SMABacktester(symbol, int(parameters[0]), int(parameters[1]), start, end, tc)
    tester1.test_strategy()

    #Bollinger
    tester2 = MeanRev.MeanRevBacktester(symbol, int(parameters[2]), int(parameters[3]),  start, end, tc)
    tester2.test_strategy()

    # Create Comb
    comb = tester1.results.loc[:, ["returns", "position"]].copy()
    comb.rename(columns = {"position":"position_SMA"}, inplace = True)
    comb["position_MR"] = tester2.results.position

    # Methode 2
    comb["position_comb"] = np.sign(comb.position_MR + comb.position_SMA)

    # Busy Hours
    comb["NYTime"] = comb.index.tz_convert("America/New_York")
    comb["hour"] = comb.NYTime.dt.hour
    comb.position_comb = np.where(comb.hour.between(2, 12), comb.position_comb, 0)

    # Backtest
    comb["strategy"] = comb["position_comb"].shift(1)*comb["returns"]
    comb.dropna(inplace = True)
    comb["trades"] = comb.position_comb.diff().fillna(0).abs()
    comb.strategy = comb.strategy - comb.trades * tc
    comb["creturns"] = comb["returns"].cumsum().apply(np.exp)
    comb["cstrategy"] = comb["strategy"].cumsum().apply(np.exp)

    return comb # -comb["cstrategy"].iloc[-1]

In [7]:
optimal_strategy((50, 150, 75, 3))

,returns,position_SMA,position_MR,position_comb,NYTime,hour,strategy,trades,creturns,cstrategy
time,,,,,,,,,,
2019-01-04 00:20:00+00:00,0.000338,-1,-1.0,0.0,2019-01-03 19:20:00-05:00,19,0.0,0.0,1.000338,1.000000
2019-01-04 00:40:00+00:00,0.000627,-1,-1.0,0.0,2019-01-03 19:40:00-05:00,19,0.0,0.0,1.000965,1.000000
2019-01-04 01:00:00+00:00,-0.000013,-1,-1.0,0.0,2019-01-03 20:00:00-05:00,20,-0.0,0.0,1.000952,1.000000
2019-01-04 01:20:00+00:00,0.000180,-1,-1.0,0.0,2019-01-03 20:20:00-05:00,20,0.0,0.0,1.001132,1.000000
2019-01-04 01:40:00+00:00,-0.000298,-1,-1.0,0.0,2019-01-03 20:40:00-05:00,20,-0.0,0.0,1.000834,1.000000
...,...,...,...,...,...,...,...,...,...,...
2020-08-30 22:20:00+00:00,0.000382,1,0.0,0.0,2020-08-30 18:20:00-04:00,18,0.0,0.0,1.045206,1.150154
2020-08-30 22:40:00+00:00,-0.000092,1,0.0,0.0,2020-08-30 18:40:00-04:00,18,-0.0,0.0,1.045109,1.150154
2020-08-30 23:00:00+00:00,-0.000143,1,0.0,0.0,2020-08-30 19:00:00-04:00,19,-0.0,0.0,1.044960,1.150154


In [9]:
from scipy.optimize import minimize

In [11]:
bnds = ((25, 75), (100, 200), (50, 100), (1, 5))
bnds

((25, 75), (100, 200), (50, 100), (1, 5))

In [13]:
start_par = (50, 150, 75, 3)

In [15]:
opts = minimize(optimal_strategy, start_par, method = "Powell", bounds = bnds)

ValueError: The user-provided objective function must return a scalar value.

In [ ]:
opts # add .x to opts to view which parameter is optimal

In [18]:
df[["creturns", "cstrategy"]].plot(figsize = (12, 8), fontsize = 12)
plt.legend(fontsize = 12)
plt.show()

KeyError: "None of [Index(['creturns', 'cstrategy'], dtype='object')] are in the [columns]"